In [1]:
import pymrio
exio3 = pymrio.parse_exiobase3(path="exiobase/IOT_2022_pxp.zip")
import country_converter as coco
cc = coco.CountryConverter()
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Suppress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
region_agg = coco.agg_conc(original_countries='EXIO3', aggregates=["EU"], missing_countries='Non EU', merge_multiple_string=None)

In [3]:
exio3.aggregate(region_agg = region_agg)

In [4]:
exio3.calc_all()
exio3_ori = exio3.copy()
Y = exio3.Y.copy()
x = exio3.x.copy()
Z = exio3.Z.copy()
A = exio3.A.copy()
L = exio3.L.copy()

In [5]:
# Setting up the variables
vehicles_list = ["ICEV","PHEV","BEV","FCEV"]
stressor = "Carbon dioxide (CO2) IPCC categories 1 to 4 and 6 to 7 (excl land use, land use change and forestry)"
#desired_order = exio3.get_sectors()
scenario_list = ["Projection-STEPS","Projection-APS"]
year_list = [2023,2025,2030,2035]
demand_list = ['normal','EU100','EU0']
column_list = ["scenario","year","vehicles","stage","region","emission"]

In [6]:
def calc_emission(vehicles,outlook): 
    exio3.Y.iloc[:,:] = 0
    exio3.Y["EU",vehicles] = outlook[vehicles]
    exio3.Y.fillna(0,inplace=True)
    
    # calculate with new demand
    # coeff : A, L, S, M should be the same with original
    exio3.x = None # reset the output
    exio3.Z = None  # expect new transaction to satisfy the new the output
    exio3.impacts.D_cba = None # reset foot print cba, others are not synch because they consider the emission from final demand
    exio3.impacts.F = None
    exio3.calc_all()
    
    # updating F with current output 
    #x = exio3.x
    #S = exio3.impacts.S
    #diagonal_x = pd.DataFrame(0.0000000, index=x.index, columns=x.index)
    #np.fill_diagonal(diagonal_x.values, x.values.flatten())
    
    #F = S.dot(diagonal_x)
    #exio3.impacts.F = F
    
    diag_emission = None
    diag_emission = exio3.impacts.diag_stressor(stressor) #become a matrix F of regional - sector X regional - sector 
    #diag_emission.F
    
    exio3.emission  = diag_emission
    exio3.calc_all()
    
    #print(exio3.meta)
    #print("done calculation for "+vehicles)
    emission = exio3.emission.D_cba
    return emission

In [7]:
def calc_emission_stage(stage):
    #start collectimg emission data
    emission_stage = pd.DataFrame(columns=column_list)

    for scenario in scenario_list:
        # read new outlook 
        file_path = "data/outlook_"+stage+"_"+scenario+".csv"
        if ((stage=="manufacturing") | (stage=="battery")): #manufacturing and battery stage will depend on where the stage will be produced
           file_path = "data/outlook_"+stage+"_"+scenario+"_"+demand+".csv"
        
        #---------------------------------
        outlook = pd.read_csv(file_path)
        outlook.set_index(keys=["year","region","sector"],inplace=True)

        for year in year_list: 
            for vehicles in vehicles_list:
                emission = calc_emission(vehicles,outlook.loc[year])
                emission_regions = emission.groupby(level=0).sum()
                emission_regions_total = emission_regions.sum(axis=1).to_frame().reset_index()
                emission_regions_total.rename(columns = {0:"emission"}, inplace=True)
                emission_regions_total["stage"] = stage
                emission_regions_total["scenario"] =  scenario
                emission_regions_total["year"] =  year
                emission_regions_total["vehicles"] =  vehicles
                
                ## add to overall record
                emission_stage = pd.concat([emission_stage,emission_regions_total],axis=0) 
    
    emission_stage.reset_index(drop=True,inplace=True) 
    emission_stage.to_csv("outputs/emission_by_stage_"+stage+"_"+demand+".csv",index=False)

In [8]:
#selected scenario - MANUFACTURING, WELL TO TANK, BATTERY REPLACEMENT, DISPOSAL
for demand in demand_list:
    calc_emission_stage(stage="manufacturing")
    calc_emission_stage(stage="welltotank")
    calc_emission_stage(stage="battery")
    calc_emission_stage(stage="disposal")

In [9]:
# CALCULATE DIRECT EMISSION 
stage="driving"
region = "EU" #all emission will be produced in EU

# emission will depend on vehicle population (not sales)
file_path = "data/outlook_vehicles_stock.csv"
outlook = pd.read_csv(file_path)
outlook = outlook[outlook.year.isin(year_list)] #only specified year data 
outlook = pd.melt(outlook, id_vars=['category','region','year'], value_vars=['ICEV', 'PHEV', 'BEV', 'FCEV'],
                    var_name='vehicles', value_name='outlook')

# requirement or assumption for direct emission
file_path = "inputs/input_direct_emission.csv"
requirement = pd.read_csv(file_path)
requirement = pd.melt(requirement, id_vars=['assumption','code','unit'], value_vars=['ICEV', 'PHEV', 'BEV', 'FCEV'],var_name='vehicles', value_name='value')

requirement = pd.pivot(requirement,index="vehicles",columns="code", values="value").reset_index()
requirement.fillna(0,inplace=True)

outlook_requirement = pd.merge(outlook, requirement, on='vehicles', how='outer')
# reminder: the assumption is in tCO2, stressor in exiobase in gigagram co2 (10^9)
outlook_requirement["emission"] = (outlook_requirement["outlook"] * outlook_requirement["vehicle_annual_mileage"] * outlook_requirement["emission_intensity"]) / 1000 # from ton to Gg
outlook_requirement.rename(columns={"category":"scenario"},inplace=True)

outlook_requirement["stage"]=stage
outlook_requirement["region"]=region

for demand in demand_list:
    emission_stage = outlook_requirement[column_list]
    #emission_stage = emission_stage[(emission_stage.scenario==scenario)]
    emission_stage.to_csv("outputs/emission_by_stage_"+stage+"_"+demand+".csv",index=False)

In [10]:
stage_list = ["manufacturing","welltotank","driving","battery","disposal"]

for demand in demand_list:
    emission_stages = pd.DataFrame()
    for stage in stage_list:
        file_path = "outputs/emission_by_stage_"+stage+"_"+demand+".csv"
        emission_stage = pd.read_csv(file_path)
        emission_stages = pd.concat([emission_stages,emission_stage])

    emission_stages.to_csv( "outputs/emission_by_stage_all_"+demand+".csv",index=False)